# Denoising Probabilistic Diffusion Models

In [ ]:
!pip install fiftyone wandb open-clip-torch

## Part 1

In [ ]:
import torch

from utils import UNet_utils, ddpm_utils

# TODO: Initialize the U-Net model and load the pre-trained weights from notebook 05.

# Make sure to use the same architecture as in the notebook.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet_utils.UNet(

    T=400, img_ch=3, img_size=32, down_chs=(256, 256, 512), t_embed_dim=8, c_embed_dim=512

).to(device)

# model.load_state_dict(torch.load('path_to_your_model.pth')) # You need to provide the path to your trained model

model.eval()

# TODO: Define a list of text prompts to generate images for.

text_prompts = [

    "A photo of a red rose",

    "A photo of a white daisy",

    "A photo of a yellow sunflower"

]

# --- Embedding Extraction using Hooks ---

# We will use PyTorch hooks to extract the output of the 'down2' layer (the bottleneck).

embeddings_storage = {}

def get_embedding_hook(name):

    def hook(model, input, output):

        embeddings_storage[name] = output.detach()

    return hook

# TODO: Register a forward hook on the `down2` layer of the U-Net model.

# The hook should store the output of the layer in the `embeddings_storage` dictionary.

# model.down2.register_forward_hook(get_embedding_hook('down2'))


# TODO: Modify the `sample_flowers` function from notebook 05 to generate images 

# and store the extracted embeddings.

# You will need to run the generation process and then access the `embeddings_storage`

# to get the embeddings for each generated image.

# generated_images, _ = sample_flowers(text_prompts)

# extracted_embeddings = embeddings_storage['down2']

## Part 2

In [ ]:
import open_clip

# TODO: Calculate the CLIP score for each generated image against its prompt.

# You can use the `calculate_clip_score` function from the evaluation guide.

# TODO: Calculate the FID score for the set of generated images.

# You will need the `calculate_fid` function and the Inception model from the evaluation guide.

# You will also need to load the real TF-Flowers dataset to compare against.

## Part 3

In [ ]:
import fiftyone as fo

import fiftyone.brain as fob

# TODO: Create a new FiftyOne dataset.

dataset = fo.Dataset(name="generated_flowers_with_embeddings")

# TODO: Iterate through your generated images and add them to the dataset.

# For each image, create a fiftyone.Sample and add the following metadata:

# - The file path to the saved image.

# - The text prompt (as a `fo.Classification` label).

# - The CLIP score (as a custom field).

# - The extracted U-Net embedding (as a custom field).

# TODO: Compute uniqueness for the dataset.

# fob.compute_uniqueness(dataset)

# TODO: Compute representativeness using the extracted U-Net embeddings.

# fob.compute_representativeness(dataset, embeddings="unet_embedding")

# TODO: Launch the FiftyOne App to visualize your dataset and analyze the results.

# session = fo.launch_app(dataset)

## Part 4

In [ ]:
import wandb

# TODO: Login to wandb.

# wandb.login()

# TODO: Initialize a new wandb run.

# run = wandb.init(project="diffusion_model_assessment_v2")

# TODO: Log your hyperparameters (e.g., guidance weight `w`, number of steps `T`).

# TODO: Log your evaluation metrics (CLIP Score and FID).

# TODO: Create a wandb.Table to log your results. The table should include:

# - The generated image.

# - The text prompt.

- The CLIP score.

# - The uniqueness score.

# - The representativeness score.

# TODO: Finish the wandb run.

# run.finish()